In [1]:
import json
import os
import math
import librosa
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [5]:
DATASET_PATH = "genres_dataset_reduced"
JSON_PATH = "data_reduced.json"
SAMPLE_RATE = 22050 #podana w datasecie
TRACK_DURATION = 30 #sekundy
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION
DATA_PATH = "data_reduced.json"


def save_mfcc(DATASET_PATH,JSON_PATH, num_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):

    data = {
        "genres": [],
        "labels": [],
        "mfcc": []
    }

    samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)

    # przeszukiwanie każdego folderu w podanym
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(DATASET_PATH)):

        # pomijanie folderu głównego
        if dirpath is not DATASET_PATH:

            # zapisywanie nazwy folderu, czyli nazwy gatunku
            semantic_label = os.path.basename(dirpath)
            data["genres"].append(semantic_label)
            print(f"\nProcessing: {semantic_label}")

            # processing plików .wav
            for filename in filenames:

                file_path = os.path.join(dirpath, filename)
                signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)

                # dzielimy plik na segmenty
                for d in range(num_segments):

                    start = samples_per_segment * d
                    finish = start + samples_per_segment

                    # wydobycie mfcc
                    mfcc = librosa.feature.mfcc(y=signal[start:finish], sr=sample_rate, n_mfcc=num_mfcc, n_fft=n_fft,
                                                hop_length=hop_length)
                    mfcc = mfcc.T

                    # zapisywanie mfcc
                    if len(mfcc) == num_mfcc_vectors_per_segment:
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(i - 1)
                        print(f"{file_path}, segment:{d+1}")

    # eksport do .json
    with open(JSON_PATH, "w") as fp:
        json.dump(data, fp, indent=4)



save_mfcc(DATASET_PATH, JSON_PATH, num_segments=10)

In [7]:
def load_data(data_path):

    with open(data_path, "r") as fp:
        data = json.load(fp)

    # konwersja donumpy arrays
    x = np.array(data["mfcc"])
    y = np.array(data["labels"])

    #print("Data succesfully loaded!")

    return  x, y

def plot_history(history):

    fig, axs = plt.subplots(2)

    axs[0].plot(history.history["accuracy"], label="train accuracy", color="blue")
    axs[0].plot(history.history["val_accuracy"], label="test accuracy", color="red")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")

    axs[1].plot(history.history["loss"], label="train error", color="blue")
    axs[1].plot(history.history["val_loss"], label="test error", color="red")
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")

    plt.show()

x, y = load_data(DATA_PATH)

# podział danych na treningowe i testowe
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

# tworzymy model MLP
model = keras.Sequential([

    # input layer
    keras.layers.Flatten(input_shape=(x.shape[1], x.shape[2])),

    # 1st dense layer
    #keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dropout(0.3),

    # 2nd dense layer
    #keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dropout(0.3),

    # 3rd dense layer
    #keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dropout(0.3),

    # output layer
    keras.layers.Dense(10, activation='softmax')
])

# kompilacja modelu
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

model.summary()

# trening
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=32, epochs=50)

plot_history(history)

ValueError: With n_samples=0, test_size=0.3 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.